In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
from sklearn.model_selection import train_test_split

In [18]:
lines= pd.read_table(r"C:\Users\Dhanashekar\Desktop\proper nouns\names1.csv", names=['eng', 'mar'])

lines.mar = lines.mar.apply(lambda x : '< '+str(x)+ ' >')

In [19]:
# collection of words with typo errors
eng_typo_errors=set()
for word in lines.eng:
    if word not in eng_typo_errors:
            eng_typo_errors.add(word)

# collection of correct words
eng_correct_words=set()
for word in lines.mar:
    if word not in eng_correct_words:
            eng_correct_words.add(word)

# Max Length of source sequence
lenght_list=[]

max_length_src = np.max(eng_typo_errors)

# Max Length of target sequence
lenght_list=[]

max_length_tar = np.max(eng_correct_words)

input_words = sorted(list(eng_typo_errors))
target_words = sorted(list(eng_correct_words))

# Calculate Vocab size for both source and target
num_encoder_tokens = len(eng_typo_errors)
num_decoder_tokens = len(eng_correct_words)
num_decoder_tokens += 1 # For zero padding

# Create word to token dictionary for both source and target
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [21]:

X_train, X_test, y_train, y_test = train_test_split(input_token_index,target_token_index, test_size=0.15)

ValueError: Found input variables with inconsistent numbers of samples: [7623, 1]

In [22]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

NameError: name 'X_train' is not defined

In [15]:
# Encoder
encoder_inputs =tf.keras.layers.Input(shape=(None,))
enc_emb =tf.keras.layers.Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Use a softmax to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

NameError: name 'latent_dim' is not defined